<a href="https://colab.research.google.com/github/componavt/differential_equations/blob/main/src/hill_equation/95_save_for_streamlit_unique_x_y.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>





# 🧬📊 Solution Archive for Gene Regulatory ODE System

**Overview:**  
This notebook computes and saves numerical solutions of a two-dimensional gene-regulatory ODE system under a wide range of parameter values. Solutions are saved to disk for later visualization in interactive apps like Streamlit.

---

**Model Equations**  
\begin{cases}
\frac{dx}{dt} = \frac{K\,x^{1/α}}{b^{1/α} + x^{1/α}} \;-\; γ_1\,x,\\[6pt]
\frac{dy}{dt} = \frac{K\,y^{1/α}}{b^{1/α} + y^{1/α}} \;-\; γ_2\,y.
\end{cases}

**Fixed parameters:**  
- b = 1.0, K = 1.0  
- Time: t ∈ [0, 1.0], N = 500 steps  
- Initial condition(s): x₀ = y₀ = b * (1 - ε), ε ≈ 1e-10

**Parameter sweeps:**  
- α ∈ {1e-9, 1e-10, … , 1e-14}  
- γ₁, γ₂ ∈ {0.0, 0.5, 1.0}  
- Solvers: RK45, DOP853, BDF

---

**What this notebook does:**  
1. For each combination of α, γ₁, γ₂, and solver:
   - Integrates the ODE system from a fixed initial condition.
   - Saves time points and trajectories x(t), y(t).
2. Each solution is stored as a dictionary with:
   - Parameters: α, γ₁, γ₂, solver name, initial values
   - Solution arrays: t, x(t), y(t)
   - Status: success or failure
3. All results are saved in a single `pickle` file:  
   📁 `ode_solutions/solutions.pkl`

---

**How to use this in Streamlit:**  
- Load the file with `pickle.load(open(...))`  
- Add controls (checkboxes, sliders) for α, γ₁, γ₂, solver  
- Filter solutions and display interactive plots of x(t), y(t)  

---

*This notebook provides a scalable, solver-agnostic framework for collecting ODE simulation results for downstream visualization and analysis.*

In [4]:
# Modified code for saving ODE solutions for Streamlit with filtering across alpha and method
# Cell 1: Parameters and library imports
import numpy as np
from scipy.integrate import solve_ivp
import os

b = 1.0
t_end = 1
K = 1.0
epsilon = 1e-5  # Minimum allowed difference for uniqueness
alpha_list = [1e-9 * (10 ** -i) for i in range(6)]  # 1e-9 to 1e-14
print("Alpha values:", alpha_list)

N = 500
t_eval = np.linspace(0, t_end, N)

all_methods = ["DOP853", "BDF", "Radau", "RK23"]  # limited for simplicity

gamma_1_list = np.arange(0, 1.01, 0.1)
gamma_2_list = np.arange(0, 1.01, 0.1)

# Initial condition shifts
initial_shifts = [-1e-3, -1e-6, 0.0, 1e-6, 1e-3]
initial_conditions = [(b * (1 + shift), b * (1 + shift)) for shift in initial_shifts]

# Folder for saving data
output_folder = "ode_solutions"
os.makedirs(output_folder, exist_ok=True)

Alpha values: [1e-09, 1.0000000000000002e-10, 1.0000000000000001e-11, 1.0000000000000002e-12, 1.0000000000000002e-13, 1.0000000000000002e-14]


In [5]:
# Cell 2: Define RHS with decay terms
def get_rhs(alpha, gamma1, gamma2):
    def rhs(t, xy):
        x = np.clip(xy[0], 1e-20, 1e20)
        y = np.clip(xy[1], 1e-20, 1e20)
        inv_alpha = 1 / alpha
        x_alpha = np.exp(np.clip(np.log(x) * inv_alpha, -700, 700))
        y_alpha = np.exp(np.clip(np.log(y) * inv_alpha, -700, 700))
        b_alpha = np.exp(np.clip(np.log(b) * inv_alpha, -700, 700))
        dxdt = K * x_alpha / (b_alpha + x_alpha) - gamma1 * x
        dydt = K * y_alpha / (b_alpha + y_alpha) - gamma2 * y
        return [dxdt, dydt]
    return rhs

In [6]:
# Cell 3: Solve and collect results with uniqueness filtering across methods and alphas
entries = []
saved_signatures = []  # store {'x': array, 'y': array}

for x0, y0 in initial_conditions:
    for gamma1 in gamma_1_list:
        for gamma2 in gamma_2_list:
            for method in all_methods:
                for alpha in alpha_list:
                    rhs = get_rhs(alpha, gamma1, gamma2)
                    try:
                        sol = solve_ivp(rhs, [0, t_end], [x0, y0], t_eval=t_eval, method=method)
                        if not sol.success:
                            continue
                        x = sol.y[0]
                        y = sol.y[1]

                        # Check uniqueness across previously saved results
                        unique = True
                        for sig in saved_signatures:
                            if np.linalg.norm(x - sig['x']) < epsilon and np.linalg.norm(y - sig['y']) < epsilon:
                                unique = False
                                break
                        if not unique:
                            continue

                        # Save signature and entry
                        saved_signatures.append({'x': x, 'y': y})
                        entries.append({
                            'alpha': alpha,
                            'gamma1': gamma1,
                            'gamma2': gamma2,
                            'method': method,
                            'x0': x0,
                            'y0': y0,
                            't': sol.t,
                            'x': x,
                            'y': y,
                        })
                    except Exception as e:
                        print(f"Error: method={method}, alpha={alpha}, gamma1={gamma1}, gamma2={gamma2}, x0={x0:.2e}: {e}")

# Cell 4: Convert to npz-friendly format
npz_data = {
    'alpha': np.array([e['alpha'] for e in entries]),
    'gamma1': np.array([e['gamma1'] for e in entries]),
    'gamma2': np.array([e['gamma2'] for e in entries]),
    'method': np.array([e['method'] for e in entries]),
    'x0': np.array([e['x0'] for e in entries]),
    'y0': np.array([e['y0'] for e in entries]),
    't': np.array([e['t'] for e in entries], dtype=object),
    'x': np.array([e['x'] for e in entries], dtype=object),
    'y': np.array([e['y'] for e in entries], dtype=object),
}

# Cell 5: Save to .npz file
import numpy as _np
_np.savez_compressed(os.path.join(output_folder, "solutions_gene_ODU.npz"), **npz_data)
print(f"Saved {len(entries)} unique solution entries to 'solutions.npz'")

Saved 2583 unique solution entries to 'solutions.npz'
